In [1]:
import sys
sys.path.insert(0,'../../backend')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import gym
import stable_baselines
from stable_baselines import A2C, ACER, ACKTR, DQN, DDPG, SAC, PPO1, PPO2, TD3, TRPO
from stable_baselines.common.env_checker import check_env
from stable_baselines.common.policies import MlpPolicy
from stable_baselines.gail import ExpertDataset, generate_expert_traj

import webotsgym as wg

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [2]:
from webotsgym.utils import exponential_decay
from webotsgym.env.reward import step_pen_exp

class MyEval(wg.WbtReward):
    def __init__(self, env, config):
        super(MyEval, self).__init__(env, config)

    def calc_reward(self):
        target_distance = self.env.get_target_distance(False)
        if target_distance < 0.1:
            return 10000 + 5000 * (1 - abs(self.env.state.speed))
        else:
            reward = 0
            reward += -2
            
            if len(self.env.distances) > 0:
                diff = self.env.distances[-1] - self.env.get_target_distance(False)
                #print("Diff: ", diff)
                
                speed_max = 0.03
                if abs(self.env.state.speed) > speed_max :
                    speed_max = abs(self.env.state.speed)
                    print("+++++++++++++++++ speed: ", speed_max)
                    
                diff_max = speed_max * 1000 / (32 * config.sim_step_every_x)
                
                if diff > diff_max:
                    print("----------------- diff: ", diff, diff_max, " > diff_max")
                # print("Speed: ", self.env.state.speed)

                diff_rew = 0.5 * ( diff  / diff_max) / abs(self.env.state.speed / speed_max)
                #if abs(diff_rew) > 3:
                   # print("========================")
                   # print("Diff_rew:" , diff_rew)
                   # print("Diff%: ", diff  / diff_max)
                   # print("Speed%: ", abs(self.env.state.speed / speed_max))
                   # print("========================")
                    
                if diff_rew > 3:
                    reward += 3
                elif diff_rew < -3:
                    reward -= 3
                else:
                    reward += diff_rew
                    
            if self.env.state.action_denied:
                reward += -5
        
            if self.env.state.touching:
                reward += -100
        return reward

    def check_done(self):
        if self.env.total_reward < -5000:
            print("reward boundary, reward: ", self.env.total_reward)
            return True
        if self.env.get_target_distance(False) < 0.1:
            print("target reached, reward: ", self.env.total_reward)
            return True
        if self.env.total_reward > 25000:
            print("Stop hacking!, reward: ", self.env.total_reward)
            return True
        return False

In [3]:
config = wg.WbtConfig()
config.world_size = 3
config.num_obstacles = 0
config.sim_mode = wg.config.SimSpeedMode.FAST
config.sim_step_every_x = 10
config.relative_action = True
config.direction_type = wg.config.DirectionType.STEERING

In [4]:
env = wg.WbtGym(train=True, 
                evaluate_class=MyEval,
#                 action_class = wg.WbtActDiscrete(config, dspeed=0.05, ddir=0.05),
                config=config)

../../backend/webotsgym/env/webotenv.py:111: UserWarning: Relative property of action class is overwritten by config.relative_action.
  warnings.warn("Relative property of action class is overwritten "


Accepting on Port:  10201


In [5]:
model_name = "3x3_reach_target_v5"
model = PPO1("MlpPolicy", env, timesteps_per_actorbatch = 5000, tensorboard_log="./{}".format(model_name))





Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Please use `layer.__call__` method instead.









Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [ ]:
while True:
    model.learn(total_timesteps=500000)
    model.save("./{}".format(model_name))


reward boundary, reward:  -5102.15402090258
----------------- diff:  0.09587537993207418 0.09375  > diff_max
reward boundary, reward:  -5008.376935909232
----------------- diff:  0.09583493180441083 0.09375  > diff_max
reward boundary, reward:  -5098.226146618448
target reached, reward:  -953.7493465240144
----------------- diff:  0.09533753398455086 0.09375  > diff_max
reward boundary, reward:  -5011.2022842597
----------------- diff:  0.09521677110368665 0.09375  > diff_max
reward boundary, reward:  -5033.1094885022985
----------------- diff:  0.10926217152282536 0.09375  > diff_max
reward boundary, reward:  -5037.957966016836
----------------- diff:  0.09901757371178477 0.09375  > diff_max
reward boundary, reward:  -5092.074545729583
reward boundary, reward:  -5042.757786364394
reward boundary, reward:  -5039.530695939796
reward boundary, reward:  -5029.096572910092
reward boundary, reward:  -5036.3197517205135
reward boundary, reward:  -5045.546452101389
target reached, reward:  -

target reached, reward:  -3374.873377159996
reward boundary, reward:  -5002.999305988625
----------------- diff:  0.1091416410078283 0.09375  > diff_max
reward boundary, reward:  -5046.626165250505
target reached, reward:  -74.6200755394706
reward boundary, reward:  -5050.215051321832
reward boundary, reward:  -5001.890387323403
reward boundary, reward:  -5001.4723881713335
reward boundary, reward:  -5050.670979021563
----------------- diff:  0.09648296477675089 0.09375  > diff_max
----------------- diff:  0.09390947941528385 0.09375  > diff_max
reward boundary, reward:  -5081.397549225317
----------------- diff:  0.09495532171818441 0.09375  > diff_max
----------------- diff:  0.09465535250516054 0.09375  > diff_max
target reached, reward:  -3939.8165717409393
reward boundary, reward:  -5009.529402398625
reward boundary, reward:  -5016.22026304573
reward boundary, reward:  -5091.326703639975
reward boundary, reward:  -5002.392176542607
----------------- diff:  0.09790443649775149 0.09

reward boundary, reward:  -5074.514024860041
reward boundary, reward:  -5051.329974612096
reward boundary, reward:  -5096.301315035938
----------------- diff:  0.09874687641237373 0.09375  > diff_max
reward boundary, reward:  -5063.18840473721
target reached, reward:  -1661.302800296745
----------------- diff:  0.0999436771750013 0.09375  > diff_max
reward boundary, reward:  -5026.725158573791
target reached, reward:  -456.53438631260605
target reached, reward:  -283.6631790865766
reward boundary, reward:  -5005.596048438363
----------------- diff:  0.11190233638182234 0.09375  > diff_max
reward boundary, reward:  -5060.272149054964
target reached, reward:  -1223.4956065437182
reward boundary, reward:  -5032.596172838401
----------------- diff:  0.09553099542553056 0.09375  > diff_max
target reached, reward:  -2969.8767076805966
----------------- diff:  0.09379539538620141 0.09375  > diff_max
reward boundary, reward:  -5003.534593555845
reward boundary, reward:  -5012.878742335046
targ

----------------- diff:  0.09976725078212914 0.09375  > diff_max
target reached, reward:  -297.57961661770304
reward boundary, reward:  -5047.643318315658
reward boundary, reward:  -5074.736456709387
reward boundary, reward:  -5012.915589715535
reward boundary, reward:  -5047.602233549946
reward boundary, reward:  -5028.175063835771
reward boundary, reward:  -5095.217502341087
target reached, reward:  -173.9430743883678
target reached, reward:  -124.31361922706652
----------------- diff:  0.09986031006605922 0.09375  > diff_max
reward boundary, reward:  -5058.438237772098
reward boundary, reward:  -5005.654229722772
reward boundary, reward:  -5038.198453330772
target reached, reward:  -39.436539438175636
reward boundary, reward:  -5088.475804096099
target reached, reward:  -125.28767961105657
target reached, reward:  -84.6883512111376
----------------- diff:  0.10911036944089247 0.09375  > diff_max
reward boundary, reward:  -5068.519677559625
----------------- diff:  0.0973389967162832

----------------- diff:  0.0976176753910637 0.09375  > diff_max
reward boundary, reward:  -5003.7719930315225
----------------- diff:  0.10406087042429679 0.09375  > diff_max
reward boundary, reward:  -5094.624332507988
target reached, reward:  -529.0095225107896
reward boundary, reward:  -5029.493138204526
reward boundary, reward:  -5048.0378121649655
----------------- diff:  0.09518017317888205 0.09375  > diff_max
target reached, reward:  -1951.1097142361398
target reached, reward:  -19.058279825039413
reward boundary, reward:  -5043.084172146491
reward boundary, reward:  -5039.835406350247
reward boundary, reward:  -5045.378715538221
reward boundary, reward:  -5063.393443878477
reward boundary, reward:  -5013.447045349573
target reached, reward:  -4192.990773118245
reward boundary, reward:  -5016.858701463566
target reached, reward:  -558.7357332082365
reward boundary, reward:  -5080.596886807592
reward boundary, reward:  -5103.23011793845
reward boundary, reward:  -5077.77171453425

...

reward boundary, reward:  -5001.977140461276
target reached, reward:  -2254.9116973224154
----------------- diff:  0.10054195773012808 0.09375  > diff_max
target reached, reward:  -4706.353823077781
target reached, reward:  -677.1589601300005
target reached, reward:  -484.99304763717043
target reached, reward:  0
target reached, reward:  0
reward boundary, reward:  -5033.093820239807
reward boundary, reward:  -5003.515498559567
target reached, reward:  -3410.4285799005083
target reached, reward:  -1427.5573346736394
reward boundary, reward:  -5102.879449896084
reward boundary, reward:  -5061.952561258131
target reached, reward:  0
target reached, reward:  0
target reached, reward:  -397.8271122748166
----------------- diff:  0.10304432490679111 0.09375  > diff_max
----------------- diff:  0.10036838908054421 0.09375  > diff_max
target reached, reward:  -390.6646502910165
----------------- diff:  0.09507799333738487 0.09375  > diff_max
----------------- diff:  0.10666762822815903 0.0937

target reached, reward:  -378.60564158787383
target reached, reward:  -280.8016280283898
----------------- diff:  0.09512509868871769 0.09375  > diff_max
target reached, reward:  -2130.999433642093
reward boundary, reward:  -5097.658768506704
target reached, reward:  -296.6642037537153
----------------- diff:  0.09535555078404356 0.09375  > diff_max
target reached, reward:  -4111.882378946552
reward boundary, reward:  -5008.489643672156
target reached, reward:  -686.0043451272107
target reached, reward:  -201.1121516649102
reward boundary, reward:  -5052.977601482923
----------------- diff:  0.10052697707892944 0.09375  > diff_max
reward boundary, reward:  -5094.576135277204
reward boundary, reward:  -5036.267537459144
target reached, reward:  -617.2008972834544
reward boundary, reward:  -5004.157956946968
target reached, reward:  -2069.144755397582
----------------- diff:  0.1837247409602043 0.09375  > diff_max
reward boundary, reward:  -5085.05054185921
reward boundary, reward:  -506

target reached, reward:  -12.177649988885022
reward boundary, reward:  -5049.449125010377
target reached, reward:  -148.9762433867186
reward boundary, reward:  -5037.103035448993
----------------- diff:  0.10022640400001914 0.09375  > diff_max
target reached, reward:  -503.54329638271054
reward boundary, reward:  -5017.519231838336
reward boundary, reward:  -5081.194120199577
reward boundary, reward:  -5093.675463990666
reward boundary, reward:  -5002.652006704463
----------------- diff:  0.09422013934254975 0.09375  > diff_max
target reached, reward:  -3623.188219669379
reward boundary, reward:  -5085.263993995175
target reached, reward:  -3358.501029483346
target reached, reward:  -738.3674241016055
----------------- diff:  0.094368092151352 0.09375  > diff_max
target reached, reward:  -355.4544459995671
target reached, reward:  -1605.1959796904657
reward boundary, reward:  -5059.454672818672
----------------- diff:  0.10286734889585292 0.09375  > diff_max
reward boundary, reward:  -

target reached, reward:  -512.6314759277333
----------------- diff:  0.09605388306011908 0.09375  > diff_max
reward boundary, reward:  -5014.068457284416
reward boundary, reward:  -5069.90429438808
----------------- diff:  0.09862658020187376 0.09375  > diff_max
----------------- diff:  0.09490053846589086 0.09375  > diff_max
reward boundary, reward:  -5004.664221647623
target reached, reward:  -3694.107881780934
reward boundary, reward:  -5043.698457866516
reward boundary, reward:  -5000.4642401821475
reward boundary, reward:  -5087.006164493838
reward boundary, reward:  -5037.101226925928
----------------- diff:  0.10599222591432347 0.09375  > diff_max
target reached, reward:  -40.32267618141792
target reached, reward:  -1213.7018928189839
----------------- diff:  0.10140451218096247 0.09375  > diff_max
target reached, reward:  -4740.51749183548
----------------- diff:  0.1022149549832797 0.09375  > diff_max
target reached, reward:  -1100.4749366466676
target reached, reward:  -196.6

----------------- diff:  0.09511387286682138 0.09375  > diff_max
reward boundary, reward:  -5019.223992703045
reward boundary, reward:  -5024.380590639605
target reached, reward:  -51.908023146162215
----------------- diff:  0.09735948048515436 0.09375  > diff_max
reward boundary, reward:  -5077.5808117807355
reward boundary, reward:  -5000.604677695908
reward boundary, reward:  -5002.426895261806
----------------- diff:  0.10059148391333686 0.09375  > diff_max
target reached, reward:  -410.1957125799326
target reached, reward:  -3207.240857844472
reward boundary, reward:  -5051.17490253046
target reached, reward:  -1556.6980330582599
target reached, reward:  -689.4777501766857
reward boundary, reward:  -5007.052571016011
----------------- diff:  0.1000101926771998 0.09375  > diff_max
reward boundary, reward:  -5000.933475393408
reward boundary, reward:  -5001.56167953776
reward boundary, reward:  -5047.695587351577
reward boundary, reward:  -5089.562484598999
----------------- diff:  